# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [271]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

# выводим первые 5 строк датафрейма
data.head(5)

# смотрим размер датафрейма
data.shape

(1000, 13)

In [272]:
# Выведите статистики по категориальным признакам
cat_stats = data.describe(include='object')
cat_stats

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [273]:
# Закодируйте все категориальные столбцы с двумя категориями
binary_columns = []
for col in data.select_dtypes(include='object').columns:
    if data[col].nunique() == 2 and col != "Purchased Bike":
        binary_columns.append(col)
        # Самая частая категория -> 1, другая -> 0
        most_frequent = data[col].mode()[0]
        data[col] = (data[col] == most_frequent).astype(int)

print("Закодированные бинарные столбцы:", binary_columns)

Закодированные бинарные столбцы: ['Marital Status', 'Gender', 'Home Owner']


Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [274]:
# Удалите остальные категориальные столбцы
non_binary_cat_cols = []
for col in data.select_dtypes(include='object').columns:
    if data[col].nunique() > 2:
        non_binary_cat_cols.append(col)

data = data.drop(columns=non_binary_cat_cols)
print("Удаленные категориальные столбцы:", non_binary_cat_cols)
print("Количество удаленных столбцов:", len(non_binary_cat_cols))

Удаленные категориальные столбцы: ['Education', 'Occupation', 'Commute Distance', 'Region']
Количество удаленных столбцов: 4


Удалите столбец `ID`, так как он по сути нечисловой.

In [275]:
# Удалите столбец ID
data = data.drop(columns=['ID'])

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [276]:
# Сформируйте матрицу объект-признак X и вектор y
X = data.drop(columns=['Purchased Bike'])
y = data['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [300]:
# Разбейте данные на тренировочную и тестовую часть
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [278]:
Xtrain.shape[0]

700

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [279]:
# Масштабирование данных
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
Xtrain = pd.DataFrame(scaler.fit_transform(Xtrain), columns=Xtrain.columns)
Xtest = pd.DataFrame(scaler.transform(Xtest), columns=Xtest.columns)

Теперь обучите логистическую регрессию на тренировочных данных

In [280]:
# Обучите логистическую регрессию
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)
model.fit(Xtrain, ytrain)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


Сделайте предсказания на тренировочных и на тестовых данных.

In [281]:
# Сделайте предсказания
ytrain_pred = model.predict(Xtrain)
ytest_pred = model.predict(Xtest)

Оцените значение accuracy на трейне и на тесте.

In [282]:
# Оцените accuracy
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(ytrain, ytrain_pred)
test_accuracy = accuracy_score(ytest, ytest_pred)

print(f"Accuracy на трейне: {train_accuracy:.3f}")
print(f"Accuracy на тесте: {test_accuracy:.3f}")

Accuracy на трейне: 0.634
Accuracy на тесте: 0.577


Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [301]:
from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=2, include_bias=True)

pf.fit(Xtrain)

Xtrain_pf = pf.transform(Xtrain)

Xtest_pf = pf.transform(Xtest)

cols_pf = pf.get_feature_names_out(Xtrain.columns)

Xtrain_pf = pd.DataFrame(Xtrain_pf, columns=cols_pf)

Xtest_pf = pd.DataFrame(Xtest_pf, columns=cols_pf)

Xtrain_pf.shape[1] - Xtrain.shape[1]

29

In [284]:
print(X.info())
print(Xtrain.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Marital Status  1000 non-null   int64
 1   Gender          1000 non-null   int64
 2   Income          1000 non-null   int64
 3   Children        1000 non-null   int64
 4   Home Owner      1000 non-null   int64
 5   Cars            1000 non-null   int64
 6   Age             1000 non-null   int64
dtypes: int64(7)
memory usage: 54.8 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Marital Status             700 non-null    float64
 1   Gender                     700 non-null    float64
 2   Income                     700 non-null    float64
 3   Children                   700 non-null    float64
 4   Home Owner        

In [285]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)

Xtrain_poly = poly.fit_transform(Xtrain)

Xtest_poly = poly.transform(Xtest)

feature_names = poly.get_feature_names_out(Xtrain.columns)

Xtrain = pd.DataFrame(Xtrain_poly, columns=feature_names, index=Xtrain.index)
Xtest = pd.DataFrame(Xtest_poly, columns=feature_names, index=Xtest.index)

In [286]:
print(Xtest)
print(Xtrain)

     Marital Status  Gender  Income  Children  Home Owner  Cars       Age  \
0               0.0     1.0  0.1875       0.8         1.0  0.50  0.578125   
1               1.0     1.0  0.3750       0.4         0.0  0.50  0.375000   
2               1.0     1.0  0.1875       0.8         0.0  0.00  0.078125   
3               1.0     0.0  0.3125       0.2         1.0  0.00  0.171875   
4               1.0     1.0  0.3750       1.0         1.0  0.50  0.281250   
..              ...     ...     ...       ...         ...   ...       ...   
295             1.0     0.0  0.4375       1.0         1.0  0.75  0.234375   
296             1.0     0.0  0.3125       0.2         1.0  0.25  0.312500   
297             1.0     1.0  0.6250       0.8         1.0  0.75  0.359375   
298             1.0     1.0  0.1875       0.2         1.0  0.00  0.281250   
299             0.0     0.0  0.0000       0.8         1.0  0.50  0.250000   

     Marital Status^2  Marital Status Gender  Marital Status Income  ...  \

In [302]:
print(f"Новая размерность тренировочных данных: {Xtrain.shape}")
print(f"Новая размерность тестовых данных: {Xtest.shape}")

Новая размерность тренировочных данных: (700, 7)
Новая размерность тестовых данных: (300, 7)


Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [312]:
lr_new = LogisticRegression()

lr_new.fit(Xtrain, ytrain)

prediction_train_new = lr_new.predict(Xtrain)
prediction_test_new = lr_new.predict(Xtest)

**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

In [313]:
accuracy_train_new = accuracy_score(ytrain, prediction_train_new)
accuracy_test_new = accuracy_score(ytest, prediction_test_new)

In [314]:
print(accuracy_train_new)
print(accuracy_test_new - test_accuracy)

0.6385714285714286
0.0033333333333332993


Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [318]:
# Заново обучите логистическую регрессию на расширенной матрице
model_poly = LogisticRegression(random_state=42, max_iter=1000)
model_poly.fit(Xtrain, ytrain)

ytrain_pred_poly = model_poly.predict(Xtrain)
ytest_pred_poly = model_poly.predict(Xtest)

train_accuracy_poly = accuracy_score(ytrain, ytrain_pred_poly)
test_accuracy_poly = accuracy_score(ytest, ytest_pred_poly)

print(f"Accuracy на трейне (полином): {train_accuracy_poly:.3f}")
print(f"Accuracy на тесте (полином): {test_accuracy_poly:.3f}")

improvement = test_accuracy_poly - test_accuracy
print(f"Улучшение accuracy на тесте: {improvement:.3f}")

Accuracy на трейне (полином): 0.646
Accuracy на тесте (полином): 0.577
Улучшение accuracy на тесте: 0.000


In [ ]:
# Посмотрите recall и accuracy на тесте с классами по умолчанию
from sklearn.metrics import recall_score

# Вариант 1: Преобразовать метки в числовые
ytest_numeric = (ytest == 'Yes').astype(int)
ytest_pred_poly_numeric = (ytest_pred_poly == 'Yes').astype(int)

recall_default = recall_score(ytest_numeric, ytest_pred_poly_numeric)
accuracy_default = accuracy_score(ytest_numeric, ytest_pred_poly_numeric)

print(f"Recall по умолчанию: {recall_default:.3f}")
print(f"Accuracy по умолчанию: {accuracy_default:.3f}")

Recall по умолчанию: 0.461
Accuracy по умолчанию: 0.577


In [322]:
# Разобъем тренировочные данные на трейн и валидацию
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

In [325]:
yval = (yval == 'Yes').astype(int)

In [330]:
BestThr = 0.3  # Это значение будет из предыдущего шага подбора порога

model_final = LogisticRegression(random_state=42, max_iter=1000)
model_final.fit(Xtrain, ytrain)

# Получаем вероятности для положительного класса (класс 1)
ytest_proba = model_final.predict_proba(Xtest)[:, 1]

# Применяем найденный порог для классификации
ytest_pred_final = (ytest_proba >= BestThr).astype(int)

# Вычисляем метрики
recall_final = recall_score(ytest_numeric, ytest_pred_final)
accuracy_final = accuracy_score(ytest_numeric, ytest_pred_final)

print(f"Recall на тесте после подбора порога: {recall_final:.3f}")
print(f"Accuracy на тесте после подбора порога: {accuracy_final:.3f}")
print(f"Использованный порог: {BestThr:.2f}")

Recall на тесте после подбора порога: 0.901
Accuracy на тесте после подбора порога: 0.567
Использованный порог: 0.30


Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [291]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 665), (210, 665))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [292]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

for thr in np.arange(0, 1, 0.01):
    # ваш код здесь

print(BestThr, RecMax, BestAcc)

IndentationError: expected an indented block after 'for' statement on line 7 (4228949202.py, line 10)

Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [ ]:
# ваш код здесь

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.